In [1]:


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# Load the dataset
file_path = "C:/Users/ghada/DL/data/train_data_from_txt.csv"
data = pd.read_csv(file_path)


In [2]:
# 1. Text Preprocessing
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]
    return ' '.join(tokens)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ghada\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ghada\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
# Apply preprocessing to questions
data['processed_question'] = data['question'].apply(preprocess_text)

# Encode labels
label_encoder = LabelEncoder()
data['encoded_target'] = label_encoder.fit_transform(data['target'])
labels = np.array(data['encoded_target'])

# 3. Tokenization and Padding
# Tokenize the text
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(data['processed_question'])
word_index = tokenizer.word_index

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(data['processed_question'])

# Pad sequences to ensure uniform length
max_length = 50  # Adjust as needed
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')



In [11]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    padded_sequences, labels, test_size=0.2, random_state=42
)

# Check shapes
print("Training Data Shape:", X_train.shape)
print("Validation Data Shape:", X_val.shape)

Training Data Shape: (12361, 50)
Validation Data Shape: (3091, 50)


In [13]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128))  # Embedding layer
model.add(Bidirectional(GRU(64, return_sequences=True)))  # Bi-GRU layer
model.add(Dropout(0.5))  # Dropout to prevent overfitting
model.add(Bidirectional(GRU(64)))  # Another Bi-GRU layer
model.add(Dense(64, activation='relu'))  # Dense layer
model.add(Dropout(0.5))  # Dropout
model.add(Dense(len(np.unique(labels)), activation='softmax'))  # Output layer

In [14]:
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [16]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 16s 29ms/step - accuracy: 0.6989 - loss: 0.7824 - val_accuracy: 0.9926 - val_loss: 0.0276
Epoch 2/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.9974 - loss: 0.0147 - val_accuracy: 0.9935 - val_loss: 0.0234
Epoch 3/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.9991 - loss: 0.0044 - val_accuracy: 0.9926 - val_loss: 0.0286
Epoch 4/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 11s 29ms/step - accuracy: 0.9988 - loss: 0.0037 - val_accuracy: 0.9903 - val_loss: 0.0565
Epoch 5/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 11s 29ms/step - accuracy: 0.9972 - loss: 0.0084 - val_accuracy: 0.9945 - val_loss: 0.0327
Epoch 6/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 12s 30ms/step - accuracy: 0.9997 - loss: 0.0019 - val_accuracy: 0.9932 - val_loss: 0.0504
Epoch 7/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 11s 29ms/step - accuracy: 0.9996 - loss: 0.0023 - val_accuracy: 0.9929 - val_loss: 0.0538
Epoch 8/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 12s 31ms/step - accuracy: 1.0000 - loss: 8

In [ ]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Test accuracy: {accuracy}")

In [ ]:
def preprocess_input_text(text, tokenizer, max_length):
    """
    Preprocess a single input text, tokenize, and pad it.
    """
    # Preprocess the text
    text = preprocess_text(text)
    # Tokenize the text
    sequence = tokenizer.texts_to_sequences([text])
    # Pad the sequence
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')
    return padded_sequence

def predict_category(text, model, tokenizer, max_length, label_encoder):
    """
    Predict the category of a given text using the trained model.
    """
    # Preprocess the input text
    padded_sequence = preprocess_input_text(text, tokenizer, max_length)
    # Predict the category probabilities
    predictions = model.predict(padded_sequence)
    # Get the category index with the highest probability
    predicted_index = np.argmax(predictions, axis=1)[0]
    # Convert the index back to the category label
    predicted_category = label_encoder.inverse_transform([predicted_index])[0]
    return predicted_category


In [ ]:
# Example questions to test the prediction function
sample_questions = [
    "What is the capital of France?",
    "How do I install Python on Windows?",
    "Who won the 2020 Olympics?",
    "Explain the theory of relativity."
]

print("Sample Predictions:")
for question in sample_questions:
    predicted_category = predict_category(question, model, tokenizer, max_length, label_encoder)
    print(f"Question: {question}")
    print(f"Predicted Category: {predicted_category}")
    print("-" * 50)